In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math

## Looking at RateBeer

In [14]:
#out of exploration out of src
beers = pd.read_csv('../../baseData/RateBeer/beers.csv')

print(beers.shape)
beers.drop_duplicates(inplace=True)
print(beers.columns)
print(beers.shape)
beers.sample(10)

(442081, 14)
Index(['beer_id', 'beer_name', 'brewery_id', 'brewery_name', 'style',
       'nbr_ratings', 'overall_score', 'style_score', 'avg', 'abv',
       'avg_computed', 'zscore', 'nbr_matched_valid_ratings',
       'avg_matched_valid_ratings'],
      dtype='object')
(442081, 14)


,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,overall_score,style_score,avg,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
219775,344694,4Bs Grizzly Brown,23612,4Bs Brewery,Brown Ale,2,NaN,NaN,2.87,5.5,2.200000,NaN,0,NaN
26262,419367,Halo Ion Cannon,27203,Halo Brewery,Grodziskie/Gose/Lichtenhainer,4,NaN,NaN,3.22,4.1,3.500000,0.049287,4,3.500
242837,353262,Bucks County Peregine IPA,19060,Bucks County Brewery,Imperial IPA,1,NaN,NaN,3.02,8.7,3.400000,NaN,0,NaN
43955,291808,Croce di Malto Rus,9948,Croce di Malto,Bitter,5,NaN,NaN,3.21,4.2,3.420000,NaN,0,NaN
150675,440567,Fox River Reel It In Session IPA,660,Fox River Brewing Co.,Session IPA,4,NaN,NaN,3.18,4.3,3.425000,-0.116649,4,3.425
219938,233409,Arvada Fresh Hop IPA,13517,Arvada Beer Company,India Pale Ale (IPA),1,NaN,NaN,2.84,6.2,3.400000,NaN,0,NaN
178472,43362,Newport Beach Leftys Cream Ale,309,Newport Beach Brewing Company,Cream Ale,1,NaN,NaN,2.43,4.5,1.900000,NaN,0,NaN
423591,104670,Davenports Special Bitter,4782,Davenports &#40;prev Highgate&#41;,Bitter,11,39.0,46.0,2.92,3.8,3.009091,NaN,0,NaN
434121,354632,Vézelay Bière Blonde Sans Gluten,15026,Brasserie de Vézelay,Specialty Grain,6,NaN,NaN,2.96,4.0,2.983333,NaN,0,NaN
162129,441732,AleSmith Decadence 2016,432,AleSmith Brewing Company,American Strong Ale,41,91.0,71.0,3.55,9.5,3.612195,NaN,0,NaN


In [15]:
#out of exploration out of src
breweries = pd.read_csv('../../baseData/RateBeer/breweries.csv')

print(breweries.shape)
breweries.drop_duplicates(inplace=True)
print(breweries.columns)
print(breweries.shape)
breweries.sample(10)

(24189, 4)
Index(['id', 'location', 'name', 'nbr_beers'], dtype='object')
(24189, 4)


,id,location,name,nbr_beers
14157,15790,"United States, California",Tent City Beer Company,9
7082,11633,Russia,Argus,4
20275,17451,England,Atom,44
23426,17490,France,Ferme-Brasserie Cordoeil,3
22472,9336,South Korea,Herzen Brau Bar,3
19784,20128,"United States, Illinois",Moody Tongue Brewing Company,22
4047,5043,Germany,Brauhaus Joh. Albrecht Soltau,9
17684,152,"United States, North Carolina",Chesapeake Bay Brewing &#40;formerly Rock Cree...,14
14016,18105,"United States, California",Progress Brewing,37
17315,14879,"United States, Minnesota",HammerHeart Brewing Company,76


In [16]:
#out of exploration out of src
users = pd.read_csv('../../baseData/RateBeer/users.csv')

print(users.shape)
users.drop_duplicates(inplace=True)
print(users.columns)
print(users.shape)
users.sample(10)

(70174, 5)
Index(['nbr_ratings', 'user_id', 'user_name', 'joined', 'location'], dtype='object')
(70174, 5)


,nbr_ratings,user_id,user_name,joined,location
13379,11,101760,smithjay,1.266232e+09,NaN
48643,1,14693,afr0byte,1.093255e+09,"United States, Vermont"
41437,2,459127,BeerGeek69,1.492250e+09,NaN
54289,2,121036,idiotbox,1.294830e+09,NaN
59200,2,283308,robw,1.381399e+09,"United States, Rhode Island"
13931,1,147317,segir52,1.324638e+09,NaN
53581,1,381236,Brian1176,1.439719e+09,"United States, New Jersey"
45043,3,335040,mlong,1.410430e+09,NaN
2777,63,77818,NewBelgium5,1.215079e+09,"United States, Pennsylvania"
61555,1,206482,emcmillan,1.342951e+09,NaN


In [17]:
#there are 121 million lines of text... thats a lot
def parse_ratings_file(file_path, limit):
    data = []
    current_block = []
    i = 0

    with open(file_path, 'r') as file:  
        for line in file:
            i += 1
            
            if i%1_000_000 == 0: print('line x million: ', i/1_000_000)
            if i > limit :
                print('Saved first ' + str(limit/1_000_000) + ' million lines')
                break
            
            line = line.strip()  # Remove leading/trailing whitespace
            
            if not line: #if empty line, '' treated as false
                data.append(current_block)
                current_block = []
                continue #next iteration of loop
            
            if ': ' in line:  
                key, value = line.split(': ', 1)  # Split by the first occurrence of ': '
                current_block.append(value)
            

    # Convert the list of lists to a DataFrame
    return pd.DataFrame(data)


ratings_df = parse_ratings_file('../../baseData/RateBeer/ratings.txt', limit = 10_000_000)

#since our txt file is in a predicatble format we can make things easy on ourselves
new_column_names = [
    'beer_name',
    'beer_id',
    'brewery_name',  
    'brewery_id',    
    'style',         
    'abv',           
    'date',          
    'user_name',     
    'user_id',       
    'appearance',    
    'aroma',         
    'palate',        
    'taste',         
    'overall',       
    'rating',        
    'text',
]

ratings_df.columns =  new_column_names
# Display the first few rows of the DataFrame
ratings_df.sample(10)


line x million:  1.0
line x million:  2.0
line x million:  3.0
line x million:  4.0
line x million:  5.0
line x million:  6.0
line x million:  7.0
line x million:  8.0
line x million:  9.0
line x million:  10.0
Saved first 10.0 million lines


,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating,text
30608,Pivovara Medvedgrad Zlatni Medvjed,11471,Pivovara Medvedgrad,2124,Pilsener,4.4,1399024800,Iznogud,296471,2,5,2,4,10,2.30,"Draught at Pivnica Medvedgrad, Zagreb. Pours g..."
273257,Central City Red Racer Imperial IPA (Red Betty),77959,Central City Brewers + Distillers,4327,Imperial IPA,9.0,1189850400,presario,5348,3,7,3,7,15,3.50,GCBF cask conditioned sample. Big bubbles. Ora...
7334,Yanjing 11º Premium Extra,12045,Beijing Yanjing Brewery Co.,2244,Pale Lager,5.5,1134903600,grant,5938,1,2,1,1,1,0.60,Appears as expected. Aroma of sweetish malt - ...
162783,Le Trèfle Noir California,329076,Le Trèfle Noir,10587,India Pale Ale (IPA),5.8,1439978400,BeerGolem,169857,4,8,4,7,16,3.90,Verre au Brouhaha (MontrÃ©al). Robe couleur mi...
466048,Shimono Loco Kolsch (Sakura Kaori no Nama),46782,Shimono Loco Beer,3543,Kölsch,nan,1277200800,Ratman197,18957,4,6,5,6,13,3.40,Bottle poured a clear yellow with a small ling...
196477,Le Corsaire La Perruche,121206,Microbrasserie Le Corsaire,9465,Fruit Beer,4.6,1463133600,rodenbach99,61913,4,7,3,7,14,3.50,Style et rÃ©sumÃ©: Blanche au sureau. Comment:...
54681,Corona Light,743,Grupo Modelo (Corona),119,Pale Lager,3.7,1071486000,whaleman,5299,2,2,2,3,5,1.40,Pale golden yellow body with piddly carbonatio...
14636,Tsingtao,730,Tsingtao Brewery,116,Pale Lager,4.8,1363518000,allanbowers,132401,4,5,3,4,8,2.40,"fizzy lager, chinese style.on a warm day this ..."
118540,Brouemont India Pale Ale,25032,Brouemont,3788,India Pale Ale (IPA),6.0,1431770400,tom10101,126758,4,7,3,7,14,3.50,On tap @ Brouemont.Appearance: Slightly hazy b...
226613,Unibroue Éphémère Framboises,50684,Unibroue &#40;Sapporo&#41;,180,Fruit Beer,5.5,1133089200,Siroy,29749,2,5,3,5,8,2.30,"Bottle. Pink/orange, bubbles-carbonation - Loo..."
